In [ ]:
import urllib.request

url_yolov3_weights = "https://pjreddie.com/media/files/yolov3.weights"
url_yolov3_cfg = "https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg"
url_coco_names = "https://raw.githubusercontent.com/pjreddie/darknet/master/data/coco.names"

urllib.request.urlretrieve(url_yolov3_weights, "yolov3.weights")
urllib.request.urlretrieve(url_yolov3_cfg, "yolov3.cfg")
urllib.request.urlretrieve(url_coco_names, "coco.names")

('coco.names', <http.client.HTTPMessage at 0x1c292d417e0>)

In [2]:
import cv2
import numpy as np
import json
import requests

In [3]:
url = 'http://localhost:3000/updateRoomCount/3'
headers = {'Content-Type': 'application/json'}
url2 = 'http://localhost:3000/getRoomCount'

In [74]:
def count_people(video_stream):
    net = cv2.dnn.readNetFromDarknet('yolov3.cfg', 'yolov3.weights')

    # Load class labels
    with open('coco.names', 'r') as f:
        classes = [line.strip() for line in f.readlines()]

    # Initialize variables
    num_people = 0

#     while True:
#         # Read the next frame from the video stream
#         ret, frame = video_stream.read()
#         if not ret:
#             break

#         frame_count += 1

        # Resize frame to a smaller size for faster processing (optional)
    frame = cv2.resize(video_stream, (1280,720))

    height, width, _ = frame.shape

    # Create blob from frame
    blob = cv2.dnn.blobFromImage(frame, 1/255.0, (416, 416), swapRB=True, crop=False)

    # Set input blob for the network
    net.setInput(blob)

    # Forward pass through the network
    layer_names = net.getLayerNames()
    output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

    outs = net.forward(output_layers)

    # Initialize variables
    boxes = []
    confidences = []
    class_ids = []

    # Process outputs
    confidence_val=0.55
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > confidence_val and class_id == 0:  # Class ID 0 represents person in COCO dataset
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # Perform non-maximum suppression to eliminate redundant overlapping boxes
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, confidence_val, 0.65)

    # Count the number of people
    num_people = len(indexes)
        
    # Create a dictionary to store the count of people
    people_count = {'count': num_people}
        
    # Convert the people_count dictionary to JSON
    json_data = json.dumps(people_count)
        
    # Send the JSON data to the backend
    response = requests.post(url, data=json_data, headers=headers)
    response2 = requests.get(url2)
        
#     Check the response from the backend
    if response.status_code == 200:
        print("People count successfully sent to the backend.")
    else:
        print("Failed to send people count to the backend.")

    if response2.status_code == 200:
        print("People count successfully recieved from the backend.")
    else:
        print("Failed to recieve people count from the backend.")
    # Draw bounding boxes and labels on the frame
    # colors = np.random.uniform(0, 255, size=(len(classes), 3))
    # for i in range(len(boxes)):
    #     if i in indexes:
    #         x, y, w, h = boxes[i]
    #         label = classes[class_ids[i]]
    #         confidence = confidences[i]
    #         color = colors[class_ids[i]]
    #         cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
    #         cv2.putText(frame, f'{label}: {confidence:.2f}', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Display the frame with bounding boxes
    # cv2.imshow('Live Feed', frame)
        
    # Display the count of people in real-time
#     print('Number of people:', num_people)
    cv2.waitKey(1)

    return num_people

In [75]:
video_stream = cv2.VideoCapture('E:\Prerak\DTU\Project\Hackathon\Video3.mp4')

fps = video_stream.get(cv2.CAP_PROP_FPS)
skip_frames = int(fps)
# print('Frames skipped: ',skip_frames)

while video_stream.isOpened():
    
    ret,frame=video_stream.read()
        
    if video_stream.get(cv2.CAP_PROP_POS_FRAMES) % skip_frames == 0:
        num_people = count_people(frame)
        print('Number of people:', num_people)
        
    if cv2.waitKey(1) & 0xFF == ord('q') or not ret:
        break
        
video_stream.release()
cv2.destroyAllWindows()

{"count": 26}
Number of people: 26
{"count": 27}
Number of people: 27
{"count": 27}
Number of people: 27
{"count": 16}
Number of people: 16
{"count": 24}
Number of people: 24
{"count": 5}
Number of people: 5
{"count": 6}
Number of people: 6
